In [1]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
from scipy.interpolate import Rbf
output_notebook()

Loading BokehJS ...

In [2]:
from pydgrid import grid

## CIGRE LV European System

### Contour plot for voltages using Bokeh

In [3]:
sys1 = grid()
sys1.read('cigre_europe_residential.json')  # Load data
sys1.pf()  
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

V_an  = np.array([item['v_an']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
V_bn  = np.array([item['v_bn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
V_cn  = np.array([item['v_cn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
Pos_x = np.array([item['pos_x'] for item in sys1.buses])
Pos_y = np.array([item['pos_y'] for item in sys1.buses])
x_min = np.min(Pos_x)
x_max = np.max(Pos_x)
y_min = np.min(Pos_y)
y_max = np.max(Pos_y)

x_margin = 150
y_margin = 100

x_min2 = x_min-x_margin
x_max2 = x_max+x_margin
y_min2 = y_min-y_margin
y_max2 = y_max+y_margin

Pos_x_e = np.hstack((x_min2, x_min2, Pos_x, x_max2, x_max2))
Pos_y_e = np.hstack((y_min2, y_max2, Pos_y, y_max2, y_min2))
V_an_e  = np.hstack((1,1, V_an, 1,1))
V_bn_e  = np.hstack((1,1, V_bn, 1,1))
V_cn_e  = np.hstack((1,1, V_cn, 1,1))
rbfi_a = Rbf(Pos_x_e, Pos_y_e, V_an_e, function='linear')
rbfi_b = Rbf(Pos_x_e, Pos_y_e, V_bn_e)
rbfi_c = Rbf(Pos_x_e, Pos_y_e, V_cn_e)

x =  np.linspace(x_min2,x_max2, 500)
y =  np.linspace(y_min2,y_max2, 500)
X, Y = np.meshgrid(x, y)
Z_a = rbfi_a(X, Y)
Z_b = rbfi_b(X, Y)
Z_c = rbfi_c(X, Y)

### Graph with obtained results

In [4]:
sys1.s_radio_scale = 0.1
sys1.s_radio_min = 5.0

sys1.bokeh_tools()

p = figure(width=600, height=800,
           title='European LV Network (CIGRE)', x_range=[-150,150], y_range=[-350, 50])

p.image(image=[Z_a], x=x_min2, y=y_min2, dw=(x_max2-x_min2), dh=(y_max2-y_min2), palette="Spectral11")


# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=1, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size='s_radio', color='s_color', alpha=1)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

### Interaction with powers

In [5]:
sys1.read('cigre_europe_residential.json')  # Load data
sys1.pf()  
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

s_0_3pn = np.copy(sys1.pq_3pn) # 
s_0_1p  = np.copy(sys1.pq_1p)  # 
sys1.bokeh_tools()
p = figure(width=800, height=400,
           title='Voltage vs load powers', 
           x_range = [40,-350], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-400,300],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-400,300],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_loads(load_factor=1.0):

    sys1.pq_1p = load_factor*s_0_1p
    sys1.pq_3pn = np.copy(load_factor*s_0_3pn)
    sys1.set_pf()
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    #sys1.bokeh_tools()
    v_an_m = np.abs(sys1.V_node[sys1.node_1_sorter])
    v_bn_m = np.abs(sys1.V_node[sys1.node_2_sorter])
    v_cn_m = np.abs(sys1.V_node[sys1.node_3_sorter])
    sys1.bus_data['v_an'] = v_an_m
    sys1.bus_data['v_bn'] = v_bn_m
    sys1.bus_data['v_cn'] = v_cn_m
    source.data = sys1.bus_data

    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [6]:
from ipywidgets import interact
interact(update_loads, load_factor=(-1,1.2, 0.1))

interactive(children=(FloatSlider(value=1.0, description='load_factor', max=1.2, min=-1.0), Output()), _dom_cl…

<function __main__.update_loads(load_factor=1.0)>

### Analysis using Pandas

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame()
df['nodes'] = sys1.nodes
df['I_node_m'] = np.abs(sys1.I_node)
df['V_node_m'] = np.abs(sys1.V_node)
#df['phi'] = np.angle(sys1.I_node, deg=True) - np.angle(sys1.V_node, deg=True)
s  = np.conjugate(sys1.I_node)*sys1.V_node
df['p_kW']  = s.real/1000
df['q_kVA'] = s.imag/1000

df

,nodes,I_node_m,V_node_m,p_kW,q_kVA
0,R0.1,12.279686,11547.000000,133.228235,48.534983
1,R0.2,12.194776,11547.000000,132.224879,48.424213
2,R0.3,12.219433,11547.000000,132.822486,47.610666
3,R1.1,294.305597,226.183736,-63.206394,-20.883865
4,R1.2,294.318022,226.377454,-63.338571,-20.673117
5,R1.3,294.268246,227.024469,-63.455017,-20.892937
6,R1.4,0.129777,0.488038,-0.000018,-0.000061
7,R11.1,22.619924,221.082177,-4.751117,-1.560611
8,R11.2,22.542696,221.818706,-4.749990,-1.562532
9,R11.3,22.431808,222.842141,-4.748898,-1.560605


## Visualizing Pandas dataframe with qgrid

In [9]:
import qgrid

qgrid.show_grid(df)

ModuleNotFoundError: No module named 'qgrid'